# Hello Image Data

This tutorial demonstrates how to train an image classifier using TensorFlow and the [Ray AI Runtime](https://docs.ray.io/en/latest/ray-air/getting-started.html).

You should be familiar with TensorFlow before starting this tutorial. If you need a refresher, read TensorFlow's [Convolutional Neural Network](https://www.tensorflow.org/tutorials/images/cnn) tutorial.

## Before you begin

* Install the [Ray AI Runtime](https://docs.ray.io/en/latest/ray-air/getting-started.html). You'll need Ray 1.13 later to run this example.

```
pip instsall 'ray[data,tune]'
```

* Install `tensorflow` and `tensorflow-datasets`

```
pip install tensorflow tensorflow-datasets
```


# Load and normalize CIFAR-10

In [20]:
import ray
from ray.data.datasource import SimpleTensorFlowDatasource
import tensorflow as tf

from tensorflow.keras import layers, models
import tensorflow_datasets as tfds

def train_dataset_factory():
    return tfds.load("cifar10", split=["train"], as_supervised=True)[0]

def test_dataset_factory():
    return tfds.load("cifar10", split=["test"], as_supervised=True)[0]

train_dataset = ray.data.read_datasource(  
    SimpleTensorFlowDatasource(), dataset_factory=train_dataset_factory
)
test_dataset = ray.data.read_datasource(SimpleTensorFlowDatasource(), dataset_factory=test_dataset_factory)


2022-05-21 16:17:06,961	WARNING read_api.py:252 -- The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_prepare_read pid=13883) 2022-05-21 16:17:06,955	WARNING tensorflow_datasource.py:53 -- `SimpleTensorFlowDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.
2022-05-21 16:17:15,176	WARNING read_api.py:252 -- The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
(_prepare_read pid=13883) 2022-05-21 16:17:15,172	WARNING tensorflow_datasource.py:53 -- `SimpleTensorFlowDatasource` doesn't support parallel reads. The `parallelism` argument will be ignored.


In [21]:
def normalize_images(batch):
    return [(tf.cast(image, tf.float32) / 255.0, label) for image, label in batch]

train_dataset = train_dataset.map_batches(normalize_images)
test_dataset = test_dataset.map_batches(normalize_images)

Read->Map_Batches: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it]


In [3]:
import pandas as pd
from ray.data.extensions import TensorArray


def convert_batch_to_pandas(batch):
    images = [image.numpy() for image, _ in batch]
    labels = [label.numpy() for _, label in batch]

    df = pd.DataFrame({"image": images, "label": labels})

    return df
    

train_dataset = train_dataset.map_batches(convert_batch_to_pandas)
test_dataset = test_dataset.map_batches(convert_batch_to_pandas)

test_dataset

Map_Batches: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


Dataset(num_blocks=1, num_rows=10000, schema={image: object, label: int64})

## Train a convolutional neural network

In [22]:
def build_model():
    model = models.Sequential()
    # def squeeze(input):
    #     print(input.shape)
    #     return tf.squeeze(input, axis=1)
    # model.add(layers.Lambda(squeeze))
    model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(84, activation='relu'))
    model.add(layers.Dense(10))
    return model

In [10]:
from ray import train
from ray.train.tensorflow import prepare_dataset_shard


# Slower than Torch?

def train_loop_per_worker(config):
    dataset_shard = train.get_dataset_shard("train")
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    with strategy.scope():
        model = build_model()
        model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
    
    for epoch in range(2):  # TODO: Change to 2 epochs
        tf_dataset = prepare_dataset_shard(
            dataset_shard.to_tf(
                feature_columns=["image"],
                label_column="label",
                output_signature=(
                    tf.TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(None, 1), dtype=tf.uint8),
                ),
                batch_size=config["batch_size"],
                unsqueeze_label_tensor=True,
            )
        )
        model.fit(tf_dataset)
        train.save_checkpoint(epoch=epoch, model_weights=model.get_weights())

In [19]:
from ray.ml.train.integrations.tensorflow import TensorflowTrainer

trainer = TensorflowTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={"batch_size": 2},
    datasets={"train": train_dataset},
    scaling_config={"num_workers": 2}
)
result = trainer.fit()
latest_checkpoint = result.checkpoint

Trial name,status,loc
TensorflowTrainer_cde45_00000,ERROR,127.0.0.1:15129
Trial name,# failures,error file
TensorflowTrainer_cde45_00000,1,/Users/balaji/ray_results/TensorflowTrainer_2022-05-21_16-14-46/TensorflowTrainer_cde45_00000_0_2022-05-21_16-14-46/error.txt


(raylet) 2022-05-21 16:14:46,887	INFO context.py:70 -- Exec'ing worker with command: exec /Users/balaji/GitHub/ray/.venv/bin/python /Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=52605 --object-store-name=/tmp/ray/session_2022-05-21_16-05-51_439235_13824/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-21_16-05-51_439235_13824/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=64222 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:56352 --redis-password=5241590000000000 --startup-token=36 --runtime-env-hash=1215741992
(raylet) 2022-05-21 16:14:52,082	INFO context.py:70 -- Exec'ing worker with command: exec /Users/balaji/GitHub/ray/.venv/bin/python /Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=52605 --object-store-name

(BaseWorkerMixin pid=15153) object
(BaseWorkerMixin pid=15153) object
(BaseWorkerMixin pid=15153) object
(BaseWorkerMixin pid=15153) object


(BaseWorkerMixin pid=15153) 2022-05-21 16:15:00.057129: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was float32, but the yielded element was [[array([[[0.2509804 , 0.2509804 , 0.24313726],
(BaseWorkerMixin pid=15153)           [0.4627451 , 0.45882353, 0.41960785],
(BaseWorkerMixin pid=15153)           [0.2509804 , 0.24705882, 0.23921569],
(BaseWorkerMixin pid=15153)           ...,
(BaseWorkerMixin pid=15153)           [0.39215687, 0.4117647 , 0.32156864],
(BaseWorkerMixin pid=15153)           [0.4117647 , 0.42352942, 0.32156864],
(BaseWorkerMixin pid=15153)           [0.24705882, 0.25882354, 0.22745098]],
(BaseWorkerMixin pid=15153) 
(BaseWorkerMixin pid=15153)          [[0.23529412, 0.23529412, 0.22745098],
(BaseWorkerMixin pid=15153)           [0.40784314, 0.4       , 0.36078432],
(BaseWorkerMixin pid=15153)           [0.2784314 , 0.2784314 , 0.25882354],

(BaseWorkerMixin pid=15152) object
(BaseWorkerMixin pid=15152) object
(BaseWorkerMixin pid=15152) object
(BaseWorkerMixin pid=15152) object
Result for TensorflowTrainer_cde45_00000:
  date: 2022-05-21_16-14-51
  experiment_id: ac2943f4b02e476aab61e6416ae944d3
  hostname: Balajis-MacBook-Pro-16-inch-2019
  node_ip: 127.0.0.1
  pid: 15129
  timestamp: 1653174891
  trial_id: cde45_00000
  


(TrainTrainable pid=15129)           [0.42352942, 0.30980393, 0.19215687]]], dtype=float32)]
(TrainTrainable pid=15129)  [array([[[1.        , 1.        , 1.        ],
(TrainTrainable pid=15129)           [1.        , 1.        , 0.98039216],
(TrainTrainable pid=15129)           [0.95686275, 0.94509804, 0.91764706],
(TrainTrainable pid=15129)           ...,
(TrainTrainable pid=15129)           [1.        , 1.        , 1.        ],
(TrainTrainable pid=15129)           [1.        , 1.        , 1.        ],
(TrainTrainable pid=15129)           [1.        , 1.        , 1.        ]],
(TrainTrainable pid=15129) 
(TrainTrainable pid=15129)          [[1.        , 1.        , 0.99215686],
(TrainTrainable pid=15129)           [0.81960785, 0.80784315, 0.78039217],
(TrainTrainable pid=15129)           [0.6392157 , 0.6039216 , 0.5647059 ],
(TrainTrainable pid=15129)           ...,
(TrainTrainable pid=15129)           [0.99215686, 0.99607843, 0.99607843],
(TrainTrainable pid=15129)           [0.9921

RayTaskError(InvalidArgumentError): [36mray::TrainTrainable.train()[39m (pid=15129, ip=127.0.0.1, repr=TensorflowTrainer)
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/tune/trainable.py", line 360, in train
    result = self.step()
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/tune/function_runner.py", line 404, in step
    self._report_thread_runner_error(block=True)
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/tune/function_runner.py", line 574, in _report_thread_runner_error
    raise e
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/tune/function_runner.py", line 277, in run
    self._entrypoint()
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/tune/function_runner.py", line 349, in entrypoint
    return self._trainable_func(
  File "/Users/balaji/GitHub/ray/.venv/lib/python3.8/site-packages/ray/ml/trainer.py", line 381, in _trainable_func
    super()._trainab

## Test the network on the test data

In [ ]:
from ray.ml.predictors.integrations.tensorflow import TensorflowPredictor
from ray.ml.batch_predictor import BatchPredictor
batch_predictor = BatchPredictor.from_checkpoint(
    checkpoint=latest_checkpoint,
    predictor_cls=TensorflowPredictor,
    model=Net(),
)
    
outputs: ray.data.Dataset = batch_predictor.predict(
    data=test_dataset, feature_columns=["image"], unsqueeze=False
)
outputs.show(1)

# Save checkpoint to file?

## What's next

TODO